# Delaunay triangulation and FSA-Net

In [1]:
import os
import re
import numpy as np
import pandas as pd
from scipy.spatial import Delaunay

#### Shoelace formula for computing the area of a polygon

In [2]:
def polygonArea(X,Y, n) :

    # Initialize area
    area = 0.0

    # Calculate value of Shoelace formula
    j = n - 1 # n is the number of points
    for i in range( 0, n) :
        area = area + (X[j] + X[i]) * (Y[j] - Y[i]) # (X[i], Y[i]) are coordinates of i'th point.
        j = i # j is previous vertex to i
    
    # Return absolute value
    return abs(area // 2.0)

### Computes dealunay triangulation for every frame of each video

In [3]:
frontal_face_df = pd.read_csv('faceMesh/frontal_face_smile_openface.csv')
frontal_face_df.columns = [column.replace(' ', '') for column in frontal_face_df.columns]
landmarks_regex = re.compile(r'^x_[0-9]+$|^y_[0-9]+$')

landmarks_locs = frontal_face_df.columns[frontal_face_df.columns.str.contains(landmarks_regex)]
landmarks = frontal_face_df[landmarks_locs].iloc[0].to_numpy(dtype=float)
landmarks_points = np.ndarray(shape=(68,2), dtype=float)

for i in range(68): # for every landmarks
    landmarks_points[i,0] = landmarks[i] # x coordinates
    landmarks_points[i,1] = landmarks[68+i] # y coordinates

tri = Delaunay(landmarks_points) # compute Delaunay triangulation
triangles = landmarks_points[tri.simplices]

TRIANGLES_SCHEME = tri.simplices

#### Saving the triangle scheme

In [4]:
# Saving the in the Iphyton-db
#%store TRIANGLES_SCHEME

# and as a json file
#import json
#variable = {}
#variable['TRIANGLES_SCHEME'] = TRIANGLES_SCHEME.tolist()
#with open('triangles_scheme.json', 'w') as f:
#    json.dump(variable, f) 

In [6]:
%store -r TRIANGLES_SCHEME
BASE_DIR = os.path.join('openFace', 'processed_interpolated', '')
landmarks_regex = re.compile(r'^x_[0-9]+$|^y_[0-9]+$')
delaunay_df_columns = ['frame'] # initialize the columns of the dataframe
delaunay_df_columns.extend([f'triangle_{i}' for i in  range(len(TRIANGLES_SCHEME))]) # initialize the columns of the dataframe

flag = True
for csv in os.listdir(BASE_DIR): # for every video

    openface_df = pd.read_csv(os.path.join(BASE_DIR, csv))
    delaunay_df_data = [] # initialize the data of the dataframe
    
    if flag:
        landmarks_locs = openface_df.columns[openface_df.columns.str.contains(landmarks_regex)]
        flag = False
    
    for index in openface_df.index: # for every frame
        delaunay_df_row_values = [index+openface_df.frame.iloc[0]] # initialize the data of the row, in this case, the value of the 'frame' column
        landmarks = openface_df[landmarks_locs].iloc[index].to_numpy(dtype=float)
        landmarks_points = np.ndarray(shape=(68,2), dtype=float)
        
        for i in range(68): # for every landmarks
            landmarks_points[i,0] = landmarks[i] # x coordinates
            landmarks_points[i,1] = landmarks[68+i] # y coordinates
                
        triangles = landmarks_points[TRIANGLES_SCHEME]
        areas = []
        for i, triangle in enumerate(triangles):
            X_coords = triangle[:,0]
            Y_coords = triangle[:,1]
            area = polygonArea(X_coords, Y_coords, 3)
            areas.append(area)
        
        # builds the dataframe
        delaunay_df_row_values.extend(areas) # build the row
        delaunay_df_row = dict(zip(delaunay_df_columns, delaunay_df_row_values)) # link the value to the column
        delaunay_df_data.append(delaunay_df_row) # append the row
        
    delaunay_df = pd.DataFrame(data = delaunay_df_data)# create the daframe with the data gathered
    
    video_name = csv.replace('_openface.csv', '')
    delaunay_df.to_csv(f'dataframes/delaunay/{video_name}_delaunay.csv', index=False)

### Testing

In [ ]:
tri = Delaunay(landmarks_points)
import matplotlib.pyplot as plt

plt.triplot(landmarks_points[:,0], landmarks_points[:,1], tri.simplices)

plt.plot(landmarks_points[:,0], landmarks_points[:,1], 'o')

plt.show()
print(len(tri.simplices))

In [ ]:
print(tri.simplices[0])

triangles = landmarks_points[tri.simplices]
for triangle in triangles:
    X = triangle[:,0]
    Y = triangle[:,1]
    area = polygonArea(X, Y, 3)
    

In [ ]:
triangle = landmarks_points[tri.simplices[0]]
X_coords = triangle[:,0]
Y_coords = triangle[:,1]
area = polygonArea(X_coords, Y_coords, 3)

In [ ]:
print(TRIANGLES_SCHEME)